# Neural Networks

In [ ]:
import os
import copy
import time
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from itertools import product
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

In [ ]:
#import dataset
#加载drive文件
from google.colab import drive

drive.mount('/content/drive') #connect the drive
dataset_file = '/content/drive/MyDrive/W-Workspace/MovieLens_da_li/dataset'
df = pd.read_csv(dataset_file + '/dataset.csv', sep=',')

Mounted at /content/drive


In [ ]:
def fix_random(seed):
    """
    Fix the random seed for reproducibility across multiple libraries.
    """
    # Set the seed for generating random numbers
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)  # For all GPUs

    # Ensure that all operations are deterministic on GPU (if used)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Initialize the seed
fix_random(42)

# Configure device
apply_pca = True  # Flag to apply PCA
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # Use CUDA if available

## Data Preprocess


In [ ]:
# Dividi il dataset in feature e target
X = df.drop(['rating'], axis=1).to_numpy()
y = df['rating'].to_numpy()

# Dividi il dataset in training, validation e test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)


#count the numebr of x_train
print("Number of train set: ", X_train.shape[0])
print("Numebr of test set: ", X_test.shape[0])
print("Number of validation set: ", X_val.shape[0])

Number of train set:  9946
Numebr of test set:  2764
Number of validation set:  1106


In [ ]:
if apply_pca:
    try:
        pca = PCA(n_components=0.95)
        pca.fit(X_train)
        X_train = pca.transform(X_train)
        X_val = pca.transform(X_val)
        X_test = pca.transform(X_test)
        print("PCA applied to the dataset")
    except Exception as e:
        print("Error occurred while applying PCA:", e)
else:
    print("PCA was not applied to the dataset")

PCA applied to the dataset


In [ ]:
# Converti i dati in tensori PyTorch
X_train = np.array(X_train, dtype=np.float32)
X_val = np.array(X_val, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)

y_train = np.array(y_train, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

val_dataloader = DataLoader(TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val)), batch_size=y_val.shape[0])
test_dataloader = DataLoader(TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test)), batch_size=y_test.shape[0])

## NN params setting


In [ ]:
import torch.nn as nn

def get_model(input_size, hidden_size=32, dropout_prob=0.2):
    model = [
        nn.Linear(input_size, hidden_size),
        nn.ReLU(),
        nn.Dropout(dropout_prob),
        nn.Linear(hidden_size, hidden_size // 2),  # 减少隐藏层单元数
        nn.ReLU(),
        nn.Dropout(dropout_prob),
        nn.Linear(hidden_size // 2, 1)             # 输出层
    ]
    return nn.Sequential(*model)

In [ ]:
from itertools import product

# 调整后的超参数
hidden_size = [128, 256]        # 减少选项
dropout_prob = [0.3]            # 减少选项，选择一个中间值
dept = [3, 4]                   # 减少模型深度的选项
epochs = 200                    # 保持不变，但可根据实际训练情况提前停止
batch_size = [16, 32]           # 减少选项，较小的批量大小可能导致训练时间增加
learning_rate = [0.001]         # 减少选项，通常较小的学习率更稳定

# 重新计算组合数
params = product(hidden_size, dropout_prob, dept, batch_size, learning_rate)
combinations = len(hidden_size)*len(dropout_prob)*len(dept)*len(batch_size)*len(learning_rate)
print("Number of combinations: ", combinations)

Number of combinations:  8


## Define function of train


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import copy
from torch.utils.tensorboard import SummaryWriter
import numpy as np

def train(model, writer, train_dataloader, val_dataloader, device, hidden_size=3,
          dropout_prob=0.2, dept=2, epochs=60, batch_size=64, learning_rate=0.001):
    # 定义损失函数和优化器
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 定义训练和验证损失列表
    train_loss = []
    val_loss = []

    # 早停设置
    best_model = None
    best_loss = np.inf
    patience = 10
    patience_counter = 0

    # 训练循环
    for epoch in range(epochs):
        epoch_start = time.time()
        epoch_loss = 0

        for x, y in train_dataloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()                    # 重置梯度
            y_pred = model(x)                        # 前向传播
            loss = criterion(y_pred, y.unsqueeze(1)) # 计算损失
            loss.backward()                          # 反向传播
            optimizer.step()                         # 更新参数
            epoch_loss += loss.item()                # 累积损失

        # 保存平均训练损失
        train_loss.append(epoch_loss / len(train_dataloader))

        # 在验证集上计算损失
        model.eval()
        epoch_val_loss = 0
        with torch.no_grad():
            for x, y in val_dataloader:
                x, y = x.to(device), y.to(device)
                y_pred = model(x)
                loss = criterion(y_pred, y.unsqueeze(1))
                epoch_val_loss += loss.item()
        val_loss.append(epoch_val_loss / len(val_dataloader))

        # 在 TensorBoard 中记录结果
        writer.add_scalar('Loss/train', train_loss[-1], epoch)
        writer.add_scalar('Loss/val', val_loss[-1], epoch)

        print(f'Epoch {epoch+1}/{epochs}, 训练损失: {train_loss[-1]:.4f}, 验证损失: {val_loss[-1]:.4f}, 时间: {time.time()-epoch_start:.2f}秒')

        # 早停判断
        if val_loss[-1] < best_loss:
            best_loss = val_loss[-1]
            best_model = copy.deepcopy(model)  # 深拷贝最佳模型
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter == patience:  # 耐心用尽，提前停止
                print("触发早停。")
                break

    print(f"训练完成，共 {epoch+1} 轮，最佳验证损失: {best_loss:.4f}")

    return best_model, train_loss, val_loss

## Define test model


In [ ]:
def test_model(model, test_dataloader, device):
    model.eval()  # 设置模型为评估模式
    y_pred_list = []
    y_true_list = []
    with torch.no_grad():  # 禁用梯度计算
        for x, y in test_dataloader:
            x, y = x.to(device), y.to(device)  # 数据移动到指定设备
            outputs = model(x).squeeze(1)  # 预测结果
            y_pred_list.append(outputs)
            y_true_list.append(y)

    # 使用 torch.cat 将列表中的所有张量连接在一起
    y_pred = torch.cat(y_pred_list).cpu().numpy()
    y_true = torch.cat(y_true_list).cpu().numpy()

    return y_pred, y_true

## Train model

In [ ]:
# 训练模型
best_model = None   # 存储最佳模型
best_loss = np.inf  # 初始化最佳损失为无穷大
best_config = None  # 存储最佳配置
iter = 0            # 初始化迭代计数器

# 遍历所有超参数组合
for hidden_size, dropout_prob, dept, batch_size, learning_rate in params:
    iter += 1
    # 打印当前迭代信息
    print(f'\nIteration {iter+1}/{combinations}')
    print(f'hidden_size: {hidden_size}, dropout_prob: {dropout_prob}, dept: {dept}, batch_size: {batch_size}, learning_rate: {learning_rate}')

    # 构建日志字符串，用于TensorBoard记录和检查模型是否已训练
    log = f'hidden_size_{hidden_size}_dropout_prob_{dropout_prob}_dept_{dept}_batch_size_{batch_size}_learning_rate_{learning_rate}'

    # 基本目录路径
    base_path = "/content/drive/MyDrive/W-Workspace/MovieLens_da_li/results/NN/"
    # 根据是否应用 PCA 选择子目录
    sub_dir = "pca" if apply_pca else "no_pca"
    # 完整的日志路径
    full_path = os.path.join(base_path, sub_dir, log)

    # 检查模型是否已训练
    if os.path.exists(full_path):
        print("Model already trained. Skipping...")  # 如果模型已训练，则跳过
    else:
        writer = SummaryWriter(full_path)

    # 创建模型实例
    model = get_model(X_train.shape[1], hidden_size=hidden_size, dropout_prob=dropout_prob)

    # 准备训练数据加载器
    train_dataloader = DataLoader(TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train)), batch_size=batch_size, shuffle=True)

    # 配置字典，用于TensorBoard记录
    config = {
        'hidden_size': hidden_size,
        'dropout_prob': dropout_prob,
        'dept': dept,
        'batch_size': batch_size,
        'learning_rate': learning_rate
    }

    # 训练模型并获取训练和验证损失
    model, train_loss, val_loss = train(model, writer, train_dataloader, val_dataloader, device, **config)

    # 在测试集上评估模型
    y_pred, y_true = test_model(model, test_dataloader, device)
    test_loss = mean_squared_error(y_true, y_pred)
    print(f'Test loss: {test_loss:.4f} - Best Test loss: {best_loss:.4f}')  # 打印测试损失和最佳测试损失

    # 将结果保存到TensorBoard
    writer.add_hparams(config, {'hparam/loss': test_loss})
    writer.flush()

    # 早停判断
    if test_loss < best_loss:
        best_loss = test_loss  # 更新最佳损失
        best_model = copy.deepcopy(model)  # 更新最佳模型
        best_config = config  # 更新最佳配置

    writer.close()  # 关闭TensorBoard写入器


Iteration 2/8
hidden_size: 128, dropout_prob: 0.3, dept: 3, batch_size: 16, learning_rate: 0.001
Epoch 1/60, 训练损失: 0.8667, 验证损失: 0.0605, 时间: 1.88秒
Epoch 2/60, 训练损失: 0.0223, 验证损失: 0.0143, 时间: 1.76秒
Epoch 3/60, 训练损失: 0.0114, 验证损失: 0.0130, 时间: 1.63秒
Epoch 4/60, 训练损失: 0.0093, 验证损失: 0.0128, 时间: 1.46秒
Epoch 5/60, 训练损失: 0.0089, 验证损失: 0.0120, 时间: 1.57秒
Epoch 6/60, 训练损失: 0.0093, 验证损失: 0.0126, 时间: 1.80秒
Epoch 7/60, 训练损失: 0.0090, 验证损失: 0.0128, 时间: 2.18秒
Epoch 8/60, 训练损失: 0.0095, 验证损失: 0.0137, 时间: 1.73秒
Epoch 9/60, 训练损失: 0.0084, 验证损失: 0.0127, 时间: 1.53秒
Epoch 10/60, 训练损失: 0.0077, 验证损失: 0.0111, 时间: 1.55秒
Epoch 11/60, 训练损失: 0.0067, 验证损失: 0.0096, 时间: 1.57秒
Epoch 12/60, 训练损失: 0.0069, 验证损失: 0.0110, 时间: 1.41秒
Epoch 13/60, 训练损失: 0.0061, 验证损失: 0.0091, 时间: 1.69秒
Epoch 14/60, 训练损失: 0.0051, 验证损失: 0.0147, 时间: 1.66秒
Epoch 15/60, 训练损失: 0.0059, 验证损失: 0.0110, 时间: 2.09秒
Epoch 16/60, 训练损失: 0.0044, 验证损失: 0.0090, 时间: 1.80秒
Epoch 17/60, 训练损失: 0.0047, 验证损失: 0.0090, 时间: 1.41秒
Epoch 18/60, 训练损失: 0.0041, 验证损失: 0.0094, 时间:

## Best loss, R2 and the best config

In [ ]:
print(f'Best config: {best_config}')
print(f'Best loss: {best_loss}')

Best config: {'hidden_size': 256, 'dropout_prob': 0.3, 'dept': 4, 'batch_size': 16, 'learning_rate': 0.001}
Best loss: 0.006481497548520565


In [ ]:
y_pred, y_true = test_model(best_model, test_dataloader, device)
r2 = r2_score(y_true, y_pred)
print(f'R2 score: {r2}')

R2 score: 0.9707600847749389
